In [1]:
import numpy as np
from hmmlearn import hmm


In [2]:
hmm_states = 2

model = hmm.CategoricalHMM(n_components=hmm_states)
model.emissionprob_ = np.array([[0.9,0.1],[0.1,0.9]])
model.transmat_ = np.array([[0.9,0.1],[0.1,0.9]])
model.startprob_ = np.array([0.4,0.6])
data,states = model.sample(n_samples = 1000,random_state=28)
data = np.squeeze(data)

In [3]:
# model2 = hmm.CategoricalHMM(n_components=hmm_states,n_iter=1000,implementation='log')
# model2.fit(data.reshape(-1,1))
# print(model2.emissionprob_,model2.startprob_,model2.transmat_)
# # print("Most likely hidden states:", hidden_states)


In [4]:
# model2.monitor_.converged

In [5]:
n_states = 2
n_features = 2
n_obs = data.shape[0]
# emission = np.random.rand(n_obs,n_states)
# emission = emission/np.tile(np.expand_dims(np.sum(emission,axis=1),axis=1),(1,2))

emission = np.random.rand(n_states,n_features)
emission = emission/np.tile(np.expand_dims(np.sum(emission,axis=1),axis=1),(1,2))
# emission = np.array([[0.96515917, 0.03484083],
#        [0.36020674, 0.63979326]])


transition =  np.random.rand(n_states,n_states)
transition = transition/np.tile(np.expand_dims(np.sum(transition,axis=1),axis=1),(1,2))
# transition = np.array([[0.48577716, 0.51422284],
#        [0.41604097, 0.58395903]])


scale_factors = np.zeros((n_obs))
forward_hat = np.zeros((n_obs,n_states))

backward_hat = np.zeros((n_obs,n_states))

init_prob = np.array([0.5,0.5])


p_old = -10000
tol = 0.01
max_iter = 100

In [6]:
print(f'emission is {emission} and transition is {transition}')

emission is [[0.43185258 0.56814742]
 [0.96066375 0.03933625]] and transition is [[0.50309097 0.49690903]
 [0.16374339 0.83625661]]


In [7]:
for ite in range(max_iter):
    print(ite)
    forward_hat = np.zeros((n_obs,n_states))
    backward_hat = np.zeros((n_obs,n_states))
    scale_factors = np.zeros((n_obs))


    forward_hat[0,:] = init_prob * emission[:,data[0]]
    scale_factors[0] = np.sum(forward_hat[0,:])
    forward_hat[0,:] = forward_hat[0,:]/scale_factors[0]
    
    for t in range(n_obs-1):
        temp = np.matmul(forward_hat[t,:] ,transition) * emission[:,data[t+1]]
        scale_factors[t+1] = np.sum(temp)
        forward_hat[t+1,:] = temp/scale_factors[t+1]
        # print(f'temp is {temp} and the scale factor is {scale_factors[t+1]} and the forward_hat is {forward_hat[t+1]}')


    backward_hat[-1,:] = scale_factors[-1]
    for t in reversed(range(n_obs-1)):
        temp = np.matmul(backward_hat[t+1,:]*emission[:,data[t+1]],transition.T)
        backward_hat[t,:] = temp/scale_factors[t]
        


    a = np.zeros((n_obs,n_states))
    b = np.zeros((n_obs,n_states,n_states))
    for i in range(n_obs):
        for j in range(n_states):
            a[i,j] = forward_hat[i,j]*backward_hat[i,j]
        temp = np.sum(a[i,:])
        a[i,:] = a[i,:]/temp

    for t in range(n_obs-1):
        for i in range(n_states):
            for j in range(n_states):
                b[t,i,j] = scale_factors[t+1]*forward_hat[t,i]*backward_hat[t+1,j]*transition[i,j]*emission[j,data[t+1]]

    for i in range(n_states):
        for j in range(n_states):
            transition[i,j] = np.sum(b[0:-1,i,j])/np.sum(b[0:-1,i,:])
            # print(np.sum(b[0:-1,i,j]),np.sum(b[0:-1,i,:]))

    for i in range(n_states):
        init_prob[i] = a[0,i]/np.sum(a[0,:])

    for i in range(n_states):
        for j in range(n_states):
            emission[j,i] = np.sum(a[np.argwhere(data==i),j]) / np.sum(a[:,j])
            
    p = np.sum(np.log(scale_factors))
    print(f'p is:{p},transition is {transition},emission is {emission},init is {init_prob}')
    # print(f'transition prob is: {transition}')
    # print(f'emission prob is:{emission}')
    if p>p_old and p - p_old < tol:
        break
    p_old = p

0
p is:-813.2131647477856,transition is [[0.65780712 0.34219288]
 [0.13444743 0.86555257]],emission is [[0.16197892 0.83802108]
 [0.92247927 0.07752073]],init is [0.9758191 0.0241809]
1
p is:-616.0378850367019,transition is [[0.82548846 0.17451154]
 [0.08588912 0.91411088]],emission is [[0.0956664  0.9043336 ]
 [0.90768446 0.09231554]],init is [9.99473438e-01 5.26561623e-04]
2
p is:-566.6570947014384,transition is [[0.92355479 0.07644521]
 [0.05075319 0.94924681]],emission is [[0.09232123 0.90767877]
 [0.88622467 0.11377533]],init is [9.99993708e-01 6.29168966e-06]
3
p is:-565.4667858921423,transition is [[0.96093384 0.03906616]
 [0.03319736 0.96680264]],emission is [[0.11188601 0.88811399]
 [0.86036673 0.13963327]],init is [9.99999950e-01 4.97275756e-08]
4
p is:-574.2635748197354,transition is [[0.97124905 0.02875095]
 [0.02763195 0.97236805]],emission is [[0.13473291 0.86526709]
 [0.8463391  0.1536609 ]],init is [1.00000000e+00 3.20991337e-10]
5
p is:-579.018466333699,transition is [

In [8]:
import sys
np.set_printoptions(threshold=sys.maxsize)
